# Procedure for Modeling a Microlensing Event

In principle microlensing events are very simple - in Point Source, Point Lens (PSPL) events, there are just three essential parameters: 
t_0: Time of closest angular separation of the lens and source
t_E: Einstein crossing time
u_0: Impact parameter, or the closest angular separation

The lightcurves of such events are perfectly smooth and symmetrical around the peak at t_0, since the magnification is purely a function of the angular separation.  

But lensing is such an exquistely sensitive technique that a number of other factors can have a measureable influence on the morphology of the lightcurve.  Some of them can have very dramatic effects, such as for a binary event where the source trajectory passes across a caustic.  But some of them can be very subtle, such as parallax, which typically causes a skew in the lightcurve for stellar events.  

Although these so-called "second order effects" add more parameters and complexity to the modeling procedure, they are very valuable, because a PSPL model tells us almost nothing about the phyiscal nature of the lensing object.  The finite extent of the source star's disk, represented by the rho parameter, and the parallax parameters pi_EN, pi_EE, for example need to be measured for us to determine the mass of the lensing object.  

In keeping with Occam's Raser though, we have to justify including additional parameters.  So the modeling procedure for an event is to start with the simplest possible model and include second order effects only when it is shown that to do so results in a measurably better fit.  This is usually evaluated by comparing the chisq of each fit.  

The usual sequence of models to fit is as follows.  Note that models that do not include parallax parameters are often referred to as 'static' models, since they neglect the movement of the Earth.  Also bear in mind that pyLIMA often uses the log10 of some parameters for computational reasons, although the parameters are presented here in their physical units. 

## Stage 1: Single lens models

We start by considering an isolated, single object as the lens, and a single source star. 

In [2]:
# Static Point Source, Point Lens
PSPL_parameters = ['t0', 'u0', 'tE']

# Static Finite Source, Point Lens:
FSPL_parameters = ['t0', 'u0', 'tE', 'rho']

# Finite Source, Point Lens with parallax:
FSPLpi_parameters = ['t0', 'u0', 'tE', 'rho', 'piEN', 'piEE']

Reasonable initial values for t_0, u_0 and t_E can be made from visual inspection of the lightcurve for the first PSPL model.  After that, the parameters of the best-fitting model in each case can be taken as the initial-guess parameters for the next, more complicated model.

For those models where the finite size of the source star is important and the rho parameter must be calculated, then limb darkening of the source star becomes a key parameter.  All stars show some degree of limb-darkening depending on their surface temperature, and the linear coefficient used to describe the change in flux that this causes depends on wavelength.  For this reason, the coefficient used to model each lightcurve depends on the filter bandpass used to gather those data.  

The coefficients are available from a paper by (Claret and Bloemen, 2011)[https://www.aanda.org/articles/aa/abs/2011/05/aa16451-11/aa16451-11.html], and should be assigned as attributes to each telescope as follows:

In [ ]:
your_event.telescopes[0].ld_gamma = 0.5

In [3]:
# Code segment [incomplete won't run - see PyLIMA examples for full function]:
#...

# Fitting a static FSPL model
fspl = FSPL_model.FSPLmodel(event)
fit1 = DE_fit.DEfit(fspl, display_progress=False, strategy='best1bin')
fit1.fit()

# Store the best-fit static FSPL model parameters
static_fspl_params = fit1.fit_results['best_model']

# Configure a dynamic FSPL model
fspl2 = FSPL_model.FSPLmodel(event, parallax=['Full',2.45935387e+06])
fit2 = DE_fit.DEfit(fspl2, display_progress=False, strategy='best1bin')

# ...give the previous best-fit parameter values as the starting guess:
fit2.model_parameters_guess = static_fspl_params
fit2.fit()

NameError: name 'FSPL_model' is not defined

After each fit, its informative to examine the residuals lightcurves - i.e. the data - the model as a function of time.  If the model fully explains the photometric signature, then the residuals will be consistent with a straight line, and the data do not justify including further parameters.  

## Binary lens models

If at this point the residuals clearly indicate signal remaining in the data that is not explained by the model so far, we should start to consider binary lens models, by including the additional parameters:

s: the projected separation of the binary lens components
q: the ratio of the masses of the binary lens components
alpha: the angle of the source's relative trajectory across the lens plane, relative to the axis of the binary.  

However, in many cases the lightcurve shows very obvious signs that it is caused by a binary lens, such as sharp discontinuities that indicate caustic crossing events.  

Either way, it's good practise to start by considering static models first (since they have the fewest parameters), then increase complexity if justified by the data.  

In [ ]:
# Note you can always check the order of the parameters as used by PyLIMA using the following:
fit.fit_parameters.keys()

# Static Uniform Source, Binary Lens
USBL_parameters = ['t0', 'u0', 'tE', 's', 'q', 'alpha']

# Dynamic Uniform Source, Binary Lens
USBL_parameters = ['t0', 'u0', 'tE', 'piEN', 'piEE', 's', 'q', 'alpha']

# Static Finite Source, Point Lens:
FSBL_parameters = ['t0', 'u0', 'tE', 'rho', 's', 'q', 'alpha']

# Finite Source, Point Lens with parallax:
FSBLpi_parameters = ['t0', 'u0', 'tE', 'rho', 'piEN', 'piEE', 's', 'q', 'alpha']

It is also informative to plot the DE_population data as a function of parameters and chisq, as this often reveals the locations of one or more minima in parameter space.  

The DE algorithm is very good at exploring parameter space but less good at quantifying the full uncertainties on each model parameter.   So we use DEfit to find the rough values of the model parameters at minima, *then do an MCMCfit starting from those parameters with the same type of model.*

In [4]:
# Code segment [incomplete won't run - see PyLIMA examples for full function]:
# Following on from fit2 above...

# Store the DE population data to a file:
fit.fit(computational_pool=pool)
np.save('fit_results.npy', fit.fit_results['DE_population'])

# Plot the DE population results:
plot_map_DE_population.map_DE_population('fit_results.npy')

NameError: name 'fit2' is not defined

Note that the original map_DE_population function was designed to explore how chisq varies as a function of binary separation s and mass ratio q, since these parameters make the most impact on the shape of the binary caustic and this is the most important plot to make.  However, the code can be adapted to plot chisq as a function of the other model parameters if you wish, just by changing the index of the data array that is used for the plots, e.g.

In [ ]:
# Original code, line 44 plots columns 4 and 5, which for a binary model corresponds to s, q values:
plt.hist2d(map_data_sort[index,4],map_data_sort[index,5],
               norm=LogNorm(),bins=(30,30))
# and also line 54, where column=-1 corresponds to the chisq value.
plt.scatter(map_data[index,4],map_data[index,5],
                c=np.log10(map_data[index,-1]),alpha=0.25)

# The first columns in this file correspond to the order of the parameters given in the model, 
# so for example in our fit2 static FSBL model, the columns [0:7] are 
# ['t0', 'u0', 'tE', 'rho', 's', 'q', 'alpha']. 

# By adjusting the column indices used (and the plot axis labels), you can explore how chisq
# varies as a function of these parameters.  

Visually inspecting the DE population plot is a great way to see if there is more than one minimum in chisq - if so, this indicates that two physically different models might explain the available data similarly well.  This is sometimes called model degeneracy, and it is quite common when modeling very complex systems like microlensing events with many inter-related parameters.  So it is important to make sure that we explore all possible minima. 

So we use the DE population plots to identify combinations of (s,q) where minima exist.  For each minima, we use MCMC to fully map the shape of each minimum.  

In [ ]:
# E.g. suppose we have performed a DEfit for a dynamic FSBL model and 
# found two minima at (s1,q1) and (s2,q2).  

# We can get most of the initial-guess parameters for our MCMC runs from the fit_results 
# for the FSBL model:
fsbl_params = fit1.fit_results['best_model']

# We will now set up two arrays of initial-guess parameters for the MCMC runs for the two
# different minima by substituting the values of (s,q) -> {(s1,q1), and (s2,q2)}:
guess1 = copy.copy(fsbl_params)
guess1[6] = s1
guess1[7] = q1

guess2 = copy.copy(fsbl_params)
guess2[6] = s2
guess2[7] = q2

# Then we configure dynamic FSPL models for the MCMC fits for both minima.  Note that this 
# is the same type of model we used for the previous DEfit.  
fsbl = FSPL_model.FSPLmodel(event, parallax=['Full',2.45935387e+06])
fit2 = MCMC_fit.MCMCfit(fsbl)
fit2.model_parameters_guess = guess1
fit2.fit()

fit3 = MCMC_fit.MCMCfit(fsbl)
fit3.model_parameters_guess = guess2
fit3.fit()

The results of the MCMC fits for all of the minima should be stored, and the best-fitting model parameters and chisq recorded in your paper draft for comparison, to see which best explains the lightcurve data.  Here again we inspect the lightcurve residuals. 

## Binary Source Models

There is one more category of models that we should consider: binary source.  Binary stars are very common in the Galaxy, so it is not unexpected that we occasionally see the effects of the source star, rather than the lensing object, having a companion in the lightcurve of the event.  Sometimes binary source lightcurves can be very similar to binary lens lightcurves, so we need to check to see which possibility fits the data best.  

PyLIMA's syntax for using a binary source model should be familiar:

In [ ]:
# dspl_parameters = [to, uo, delta_to, delta_uo, tE, q_fluxr_1, q_fluxr2, ...] 
# where q_fluxr_* is the flux ratio in each observing band.
dspl = DSPL_model.DSPLmodel(your_event)

# And the same syntax for the fitting the model can be used with any of the usual fitting 
# routines:
fit4 = MCMC_fit.MCMCfit(dspl)

## Reevaluating the photometric uncertainties

Once we have a model that provides a good fit for the data, it is important to take a step back and examine the quality of our data.  Real-world photometry has many sources of noise, including systematic correlated (or "red") noise which is not well described by the quoted uncertainties on the flux measurements produced by most data pipelines.  

Spuriously outlying datapoints with under-estimated uncertainties can cause problems when we compare models like this, because they have a disproportional impact on the resulting chisq, and sometimes cause the fitting process to erroneously fit poor data instead of higher quality points.  Sometimes we avoid this by removing obviously-poor quality points at the data reduction stage, but despite this, outlyers are not always obvious and the uncertainties are still likely to be under-estimated.  

Therefore it is common practise to re-evaluate all datapoints in the event lightcurve by comparing it to the model, *but only after a well-fitting model has been identified!*

PyLIMA provides options to include the error rescaling during the fitting process, once a good-fit model has been found.  The procedure is as follows:

In [ ]:
# This option will compute a log_10(k) for each telescopes in the chain, where
# sig_' = 10^(log_10(k)) x sig  and sig represents the flux error for each telescope.
refined_fit = MCMC_fit.MCMCfit(mymodel, rescale_photometry=True)
